# Filter

## visualize the effect of filters

### low pass  /  high pass  / band pass

### define interactive a time domain signal and get the amplitude spectrum in the frequency domain with the FFT algorithm. Select one of the filter in the filter menu, adjust it in any way and see the filter effect on the original signal.

In [ ]:
# resourcen
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

#set backend for interactive toolbar
%matplotlib nbagg 

In [ ]:
#presets
MAX_Amplitude = 5
MAX_Frequenz = 1000
MAX_Phase = 180

fNoise = 780
timeBase = np.arange(0.0, 3.0, 1/10000)     # time vector from  0...3 s, sample rate:  1/fsample (default = 10kHz)

In [ ]:
# define functions
#
def getSinus(amp,f,phase,time):
    return(amp*np.sin(2*np.pi*f*time + phase))

#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.square.html
def getRect(amp,f,time):
    return(amp*signal.square(2*np.pi*f*time))

#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.sawtooth.html#scipy.signal.sawtooth
def getTri(f,time):
    return(signal.sawtooth(2*np.pi*f*time, width=0.5))


def calc_FFT(fSample, signal):
    #https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fft.html#numpy.fft.fft
    # calc fft with numpy.fft - routines
    #
    # calculate discrete fourier transform of given signal
    fftSig = np.fft.fft(signal)
    #
    # number of points of the fft-signal
    # np.arange(0.0, 1.0 , 1/fs)
    n = len(fftSig)
    freq = np.fft.fftfreq(n, 1/fSample)
    #
    # select ineresting part 0 .. n
    # shift the zero-frequency component to the center of the spectrum.
    Y1_shift = np.fft.fftshift(fftSig)
    F1_shift = np.fft.fftshift(freq)  #Return the Discrete Fourier Transform sample frequencies.
    #
    # calc the index of the zero point
    #
    iZero = int(np.ceil(n/2.0))
    #
    # get the amplitudes and frequencies from 0 ... n/2
    Y1_pos = Y1_shift[iZero:-1]
    F1_pos = F1_shift[iZero:-1]
    #
    # normalize amplitude with (2* 1/n)
    #
    reSpectrum = 2 * 1/n * np.abs(Y1_pos)
    return(reSpectrum , F1_pos)
#

In [ ]:
# compute filters
#
# https://scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html
def LowPass_filter(fSample, fCutOff, order):
    nyq = 0.5 * fSample
    normCutOff = fCutOff/nyq
    b,a = signal.butter(order,normCutOff,btype="low")
    return(b,a)
    
def HighPass_filter(fSample, fCutOff, order):
    nyq = 0.5 * fSample
    normCutOff = fCutOff/nyq
    b,a = signal.butter(order,normCutOff,btype="high")
    return(b,a)
    
def BandPass_filter(fSample, order, fCutOff_1, fCutOff_2):
    nyq = 0.5 * fSample
    low = fCutOff_1 / nyq
    high = fCutOff_2 / nyq
    b,a = signal.butter(order, [low, high], btype="band")
    return(b,a)

In [ ]:
#interactive controls

# signal menu
#
signalForms = ['sinusoidal', 'rectangular', 'triangular']
signal_dropdown = widgets.Dropdown(description='select signal type', options=signalForms, value='sinusoidal')
#Sliders
amp_1 = widgets.FloatSlider(min=0, max=MAX_Amplitude, value=1, description="$\hat{y}$_1:")
frq_1 = widgets.FloatSlider(min=0, max=MAX_Frequenz, value=10, description="$f$_1: in Hz")
phase_1 = widgets.FloatSlider(min=0, max=MAX_Phase, value=0, description='$\phi$_1: in °')
#
amp_2 = widgets.FloatSlider(min=0, max=MAX_Amplitude, value=1, description="$\hat{y}$_2:")
frq_2 = widgets.FloatSlider(min=0, max=MAX_Frequenz, value=10, description="$f$_2: in Hz")
phase_2 = widgets.FloatSlider(min=0, max=MAX_Phase, value=0, description='$\phi$_2: in °')
#
fSample = widgets.IntSlider(min=100, max=100000, value=10000, step=10, description='sample frequency')
#
#checkbox for noise overlay
chkNoise = widgets.Checkbox(description='add noise')

# filter menu
#
filterTypes = ['low-pass','high-pass','band-pass']
filter_dropdown = widgets.Dropdown(description='select filter type', options=filterTypes, value='low-pass')
#
fCut_1 = widgets.IntSlider(min=1, max=fSample.value, value=10,step=5, description='cut-off frequency')
fCut_2 = widgets.IntSlider(min=1, max=fSample.value, value=100, step=5, description='(bandpass): high cut-off frequency')
#
#edit field for filter order
filtOrder = widgets.IntSlider(min=0, max=64, value=3, description='filter order:')
#
# reset button
BtnReset = widgets.Button(description='reset',tooltip='set to default')

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 9))
#
# 1. create a callback which updates the plot when a control-value has changed
def update_view(*args):
    
    # update timeBase 
    timeBase = np.arange(0, 3, 1/fSample.value)     # vector for time base 0...3 s, sample rate:  1/fsample (default = 10kHz)
       
    #
    #sample frequency must be at least 2 x of highest frequency in the signal
    #(Nyquist - Theorem)
    # if not: IndexError in fft-calculation
    #
    if ((frq_1.value > 50) | (frq_2.value > 50)):
        fSample.min = 2* frq_1.value
        if (frq_2.value > frq_1.value):
            fSample.min = 2* frq_2.value
    if (chkNoise.value == True):        
        fSample.min = 2* np.max([frq_1.value, frq_2.value, fNoise])
    
    # read dropdowns
    #
    signaltype = signal_dropdown.value
    filtertype = filter_dropdown.value
    
    #----------------------------------------------------------------
    # create original-signal according signal type and signal settings
    #
    if (signaltype == "sinusoidal"):
        amp_2.disabled = False
        frq_2.disabled = False
        phase_2.disabled = False
        chkNoise.disabled = False

        ySin1 = getSinus(amp_1.value, frq_1.value, phase_1.value, timeBase)
        ySin2 = getSinus(amp_2.value, frq_2.value, phase_2.value, timeBase)
        ySum = ySin1 + ySin2
        if (chkNoise.value == True): # nois signal has a fix frequency of 780Hz and an amplitude of 5
            ySum = ySin1 + ySin2 + (5*np.sin(2*np.pi*fNoise*timeBase))
    
    if (signaltype == "rectangular"):
        yRect1 = getRect(amp_1.value, frq_1.value, timeBase)
        # disable second signal controls
        #
        amp_2.disabled = True
        frq_2.disabled = True
        phase_2.disabled = True
        chkNoise.disabled = True
        #
        ySum = yRect1
        #
    if (signaltype == "triangular"):
        yTri = getTri(frq_1.value, timeBase)
        # disable second signal controls
        #
        amp_2.disabled = True
        frq_2.disabled = True
        phase_2.disabled = True
        chkNoise.disabled = True
        #
        ySum = yTri
    
    #---------------------------------------------------------------------------
    # compute selected filters and apply to signal
    #
    # deaktivate controls if necessary
    if (filtertype != 'band-pass'):
        fCut_2.disabled = True
        
    if (filtertype == "low-pass"):
        #print("filter parameters {} fSample, {} fCutOff, {} order".format(fSample.value, fCut_1.value, filtOrder.value))
        tp_b, tp_a = LowPass_filter(fSample.value, fCut_1.value, filtOrder.value)        
        #print("filter parameters {} b, {} a".format(tp_b, tp_a))
        s = signal.lfilter(tp_b, tp_a, ySum)
        w, h = signal.freqz(tp_b, tp_a, worN=10000)
        fResponse = 0.5 * fSample.value * w/np.pi
    if (filtertype == "high-pass"):
        hp_b, hp_a = HighPass_filter(fSample.value, fCut_1.value, filtOrder.value)
        s = signal.lfilter(hp_b, hp_a, ySum)
        w, h = signal.freqz(hp_b, hp_a, worN=10000)
        fResponse = 0.5 * fSample.value * w/np.pi
    if (filtertype == "band-pass"):
        fCut_2.disabled = False
        bp_b, bp_a = BandPass_filter(fSample.value, filtOrder.value, fCut_1.value, fCut_2.value)
        s = signal.lfilter(bp_b, bp_a, ySum)
        w, h = signal.freqz(bp_b, bp_a, worN=10000)
        fResponse = 0.5 * fSample.value * w/np.pi

    # compute FFT 
    #
    # normalized spectrum of amplitude
    #
    fftSig, frqSig = calc_FFT(fSample.value, ySum)
    #
    # normalized spectrum of filtered signal
    #
    fftFilt, frqFilt = calc_FFT(fSample.value, s)
    #
    # reverse transformation 
    # (use complex fft signal here)
    reOrig = np.fft.ifft(np.fft.fft(ySum))
    reSig = np.fft.ifft(np.fft.fft(s))
    
    #------------------------------------------------------
    # update the figures
    #
    #
    # limit shown time range to 3 signal periods
    if (frq_1.value < 2):
        xEnd = 3
    if (frq_1.value > 1):
        xEnd = 3* 1/frq_1.value
    if (signaltype == 'sinusoidal'): 
        if ((frq_1.value > 0) & (frq_2.value > 0) & (frq_1.value > frq_2.value)):
            xEnd = 3* 1/frq_2.value    
    #
    #limits for FFT-diagrams
    xFFTEnd = len(fftSig) -1
    add_f = -1
    peaks,_ = signal.find_peaks(fftSig, height=(0.01, 10))
    if (np.count_nonzero(peaks) > 0):
        fftError = False
        xFFTEnd = np.max(peaks)
        add_f = (len(fftSig) - xFFTEnd)-1
        if add_f > 100:
            add_f = 100


    # fig_1: signal im time domain
    #
    axes[0, 0].clear()
    axes[0, 0].set_title("signal in time domain")
    axes[0, 0].plot(timeBase, ySum, color='C0',label=r"$y(t)_{{{}}},f:{{{}}}Hz$".format(signaltype,frq_1.value))
    axes[0, 0].set_xlabel("time /s")
    axes[0, 0].set_ylabel("amplitude")
    axes[0, 0].set_xlim(timeBase[0], xEnd)
    axes[0, 0].legend(loc="best")
    axes[0, 0].grid(True)
    
    # fig_2: fft of origin signal
    #
    axes[0, 1].clear()
    axes[0, 1].set_title("FFT of original signal")
    axes[0, 1].plot(frqSig, fftSig, color='C1')
    axes[0, 1].set_xlabel("f /Hz")
    axes[0, 1].set_ylabel("amplitude")
    axes[0, 1].set_xlim(frqSig[0], frqSig[xFFTEnd+add_f])
    axes[0, 1].grid(True)
    
    # fig_3: inverse_fft original_fft
    #
    axes[0, 2].clear()
    axes[0, 2].set_title("retransformed from original-FFT")
    axes[0, 2].plot(timeBase[:len(reOrig.real)], reOrig.real, color='C4', label="retransformed\n from FFT_1" )
    axes[0, 2].set_xlabel("time /s")
    axes[0, 2].set_ylabel("amplitude")
    axes[0, 2].set_xlim(timeBase[0], xEnd)
    axes[0, 2].legend(loc='best')
    axes[0, 2].grid(True)
    
    
    # fig_4: filtered signal
    #
    # get highest frequency from fftt:
    #  - limit X-axis of FFT-Diagrams
    #  - highest frq. found in fft + 100Hz
    #
    xLim4 = len(frqFilt)
    add4 = 0
    peaks2, _ = signal.find_peaks(fftFilt, height=(0.5, 10))
    if (np.count_nonzero(peaks2) > 0):
        fftError = True
        xLim4 = np.max(peaks)
        add4 = (len(frqFilt) - xLim4) -1
        if add4 > 100:
            add4 = 100
    # fig4: filtered signal
    #
    axes[1, 0].clear()
    axes[1, 0].set_title("filtered signal in time domain")
    axes[1, 0].plot(timeBase,s, color='C2', label="filter: {}".format(filtertype))
    axes[1, 0].set_xlabel("time /s")
    axes[1, 0].set_ylabel("amplitude")
    axes[1, 0].set_xlim(timeBase[0], xEnd)
    axes[1, 0].legend(loc='best')
    axes[1, 0].grid(True)

    # fig_5: fft of filtered signal
    #
    axes[1, 1].clear()
    axes[1, 1].set_title("FFT of filtered signal")
    axes[1, 1].plot(frqFilt, fftFilt, color='C3')
    axes[1, 1].set_xlabel("f /Hz")
    axes[1, 1].set_ylabel("amplitude")
    if (xLim4+ add4) == len(frqFilt):
        axes[1, 1].set_xlim(frqFilt[0], frqFilt[-1])
    else:
        axes[1, 1].set_xlim(frqFilt[0], frqFilt[xLim4+add4])
    axes[1, 1].grid(True)

    # fig_6: inverse_fft filtered_fft 
    #
    axes[1, 2].clear()
    axes[1, 2].set_title("retransformed from filtered-FFT")
    axes[1, 2].plot(timeBase[:len(reSig.real)], reSig.real, color='C5', label="retransformed\n from FFT_2" )
    axes[1, 2].set_xlabel("time /s")
    axes[1, 2].set_ylabel("amplitude")
    axes[1, 2].set_xlim(timeBase[0], xEnd)
    axes[1, 2].legend(loc='upper center')
    axes[1, 2].grid(True)
    
    # fig_7: frequency response of filter
    #
    axes[2, 0].clear()
    axes[2, 0].set_title("Filter Frequency Response")
    axes[2, 0].plot(fResponse, np.abs(h), 'b')
    axes[2, 0].plot(fCut_1.value, 0.5*np.sqrt(2), 'ko')
    axes[2, 0].set_xlabel("frequency /Hz")
    axes[2, 0].set_ylabel("amplitude")
    axes[2, 0].axvline(fCut_1.value, color='k')
    axes[2, 0].set_xlim(0, fCut_1.value * 5)
    axes[2, 0].grid(True)
    if filtertype == "band-pass":
        axes[2, 0].axvline(fCut_2.value, color='k')
        axes[2, 0].plot(fCut_2.value, 0.5*np.sqrt(2), 'ko')
        axes[2, 0].set_xlim(0, fCut_2.value * 5)
      
    axes[2, 1].set_axis_off()
    axes[2, 2].set_axis_off()
    fig.tight_layout()
    
def reset_all(*args):
    signal_dropdown.value = 'sinusoidal'
    filter_dropdown.value = 'low-pass'
    fSample.value = 10000
    amp_1.value = 1
    amp_2.value = 1
    frq_1.value = 10
    frq_2.value = 10
    phase_1.value = 0
    phase_2.value = 0
    chkNoise.value = False
    fCut_1.value = 50
    fCut_2.value = 200
    filtOrder.value = 3

    
#--------------------------------------------------------
# 2. register the callback by using the 'observe' method
signal_dropdown.observe(update_view, 'value')
filter_dropdown.observe(update_view, 'value')
fSample.observe(update_view,'value')
amp_1.observe(update_view,'value')
amp_2.observe(update_view,'value')
frq_1.observe(update_view,'value')
frq_1.observe(update_view,'value')
phase_1.observe(update_view,'value')
phase_2.observe(update_view,'value')
chkNoise.observe(update_view,'value')
fCut_1.observe(update_view,'value')
fCut_2.observe(update_view,'value')
filtOrder.observe(update_view, 'value')
BtnReset.on_click(reset_all)

#--------------------------------------------------------
# run app
#
# force diagram drawing
update_view()
#
#
# stack the widgets using VBox/HBox
# box-layout
box_layout = widgets.Layout(display='inline-flex',flex_flow='column',align_items='flex-start',width='90%')

signal_menu = widgets.VBox([widgets.HBox([signal_dropdown, chkNoise]), fSample, widgets.HBox([amp_1, amp_2]),\
                            widgets.HBox([frq_1, frq_2]),widgets.HBox([phase_1, phase_2])], layout = box_layout)
filter_menu = widgets.VBox([filter_dropdown, widgets.HBox([fCut_1, filtOrder]), fCut_2], layout = box_layout)

widgets.VBox([widgets.HTML(value="<h2>signal menu:</h2><br>"),signal_menu,\
              widgets.HTML(value="<h2>filter menu:</h2><br>"),filter_menu, BtnReset])